## General

In [ ]:
DEGREE_NAME = 'nombre_del_grado'

In [ ]:
UNIVERSITY_NAME = 'nombre_de_la_universidad'

In [ ]:
DEGREE_URL 'url_del_grado'

In [1]:
DEGREE_COLUMNS = ['code',
'university',
'name',
'summary',
'area',
'years',
'num_credits',
'num_places',
'faculty',
'url',
'price_1',
'price_2',
'price_3',
'price_4',
'total_min_price',
'admission_mark',
'profile',
'opportunities',
'learning_mode',
'mobility_program',
'working_practices',
'basic_skills',
'specific_skills',
'transversal_skills',
'double_degree',
'schedule',
'subjects',
'languages',
'mentions'
]

In [ ]:
SUBJECT_COLUMNS = [
    'code',
    'name',
    'subject_type',
    'num_credits',
    'period',
    'course',
    'language',
    'content',
    'mention',
    'url',
    'bibliography'
]

In [2]:
subject_types = 'Formación Básica, Obligatoria, Optativa, Obligatoria-Optativa, Trabajo de Fin de Grado'

periods = {
    0: 'Anual',
    1: 'Primer Semestre',
    2: 'Segundo Semestre'
}

learning_modes = 'Presencial, Online, A distancia, Semipresencial'

## Leer PDF

Un ejemplo simple de qué herramienta usar para leer pdfs.

> Nota: aún no tengo ningún ejemplo haciendo a scrapping a un grado con todas sus asignaturas en pdf, lo incluiré aquí cuando lo tenga.

In [111]:
url = 'your_pdf.pdf'

### Usando tabula

In [ ]:
import tabula

df = tabula.read_pdf(url, pages='all', stream=True)
df

### Usando requests + pdftotext

In [113]:
import requests
import pdftotext

def read_subject_pdf(url):
    page = requests.get(url)
    subject_text = ''

    open('subject.pdf', 'wb').write(page.content)
    
    with open("subject.pdf", "rb") as f:
        pdf = pdftotext.PDF(f)
        
    for page in pdf:
        subject_text += page

    return subject_text

In [114]:
def find_between(s, first, last):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [115]:
subject_text = read_subject_pdf(url)

In [116]:
contenidos = find_between(subject_text, 'CONTENIDOS', 'METODOLOGÍA DOCENTE Y ACTIVIDADES FORMATIVAS')

In [ ]:
print(contenidos)

In [ ]:
print(subject_text)

## Leer web

### Usando requests + beautifulsoup

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

subjects_list = []

In [ ]:
def get_subject(url):
    page = requests.get(url, verify=False)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    name = soup.select('.assignaturaInfo > h2')[0].getText(strip=True)
    code = soup.select('.codi_assignatura')[0].getText().split(': ')[1]
    num_credits = int(soup.select('.credits')[0].getText().split(': ')[1])
    
    subject = soup.select('.relacions > table > tbody > tr > td')[1].getText(strip=True)
    course = int(soup.select('.relacions > table > tbody > tr > td')[2].getText())
    period = int(soup.select('.relacions > table > tbody > tr > td')[3].getText())
    subject_type = SUBJECT_TYPE[subject] or '?'

    languages = []

    if soup.select('.idiomes > dl > dd')[1].getText(strip=True) != 'No':
        languages.append('Inglés')
    
    if soup.select('.idiomes > dl > dd')[2].getText(strip=True) != 'No':
        languages.append('Catalán')
    
    if soup.select('.idiomes > dl > dd')[3].getText(strip=True) != 'No':
        languages.append('Español')

    language = ', '.join(languages)
    content = soup.select('.continguts')[0].getText(strip=True)
    bibliography = soup.select('.bibliografia')[0].getText(strip=True)
    
    subject_data = {
     'code': code,
     'name': name,
     'subject_type': subject_type,
     'num_credits': num_credits,
     'period': period,
     'course': course,
     'language': language,
     'content': content,
     'mention': '',
     'url': url,
     'bibliography': bibliography
    }
    
    subjects_list.append(subject_data)